In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf
from tensorflow import keras

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32
img_height = 224
img_width = 224

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 50947 files belonging to 7 classes.


2022-09-06 09:06:36.631857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 09:06:36.641002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 09:06:36.643584: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-06 09:06:36.647023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.


In [6]:
base_model = keras.applications.EfficientNetB4(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_s, validation_data=validation_s, epochs=3)

Epoch 1/3


2022-09-06 09:07:20.497653: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 2118s 169ms/step - loss: 1.4155 - accuracy: 0.4555 - val_loss: 1.3208 - val_accuracy: 0.5049
Epoch 2/3
12495/12495 [==============================] - 2118s 170ms/step - loss: 1.3343 - accuracy: 0.4916 - val_loss: 1.2768 - val_accuracy: 0.5164
Epoch 3/3
12495/12495 [==============================] - 2116s 169ms/step - loss: 1.3142 - accuracy: 0.4997 - val_loss: 1.2677 - val_accuracy: 0.5251


In [11]:
for layer in base_model.layers:
    layer.trainable = True

In [12]:
es = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

In [13]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=10, callbacks=[es])

Epoch 1/10
12495/12495 [==============================] - 9534s 762ms/step - loss: 0.7754 - accuracy: 0.7129 - val_loss: 0.5772 - val_accuracy: 0.7928
Epoch 2/10
12495/12495 [==============================] - 9490s 759ms/step - loss: 0.5132 - accuracy: 0.8127 - val_loss: 0.5155 - val_accuracy: 0.8228
Epoch 3/10
12495/12495 [==============================] - 9492s 760ms/step - loss: 0.3833 - accuracy: 0.8602 - val_loss: 0.5213 - val_accuracy: 0.8308
Epoch 4/10
12495/12495 [==============================] - 9485s 759ms/step - loss: 0.2997 - accuracy: 0.8902 - val_loss: 0.5694 - val_accuracy: 0.8302
Epoch 5/10
12495/12495 [==============================] - 9517s 762ms/step - loss: 0.2408 - accuracy: 0.9111 - val_loss: 0.6265 - val_accuracy: 0.8291


In [15]:
model.save('0906_EffiB4.h5')

In [16]:
model.evaluate(validation_s)

1593/1593 [==============================] - 229s 144ms/step - loss: 0.5155 - accuracy: 0.8228


[0.5154721140861511, 0.8227570056915283]

In [17]:
model.evaluate(training_s)

12495/12495 [==============================] - 1836s 147ms/step - loss: 0.3359 - accuracy: 0.8814


[0.3358798921108246, 0.8814398050308228]

In [19]:
md = keras.models.load_model('0906_EffiB4.h5')

In [21]:
md.evaluate(validation_s)

1593/1593 [==============================] - 237s 147ms/step - loss: 0.5155 - accuracy: 0.8228


[0.5154725909233093, 0.8227570056915283]